<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [2]:
import pandas as pd
yelp = pd.read_json('./data/review_sample.json', lines=True)

In [3]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [4]:
yelp['text'].head(10)

0    BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...
1    Came here for lunch Togo. Service was quick. S...
2    I've been to Vegas dozens of times and had nev...
3    We went here on a night where they closed off ...
4    3.5 to 4 stars\n\nNot bad for the price, $12.9...
5    Tasty, fast casual Latin street food.  The men...
6    This show is absolutely amazing!! What an incr...
7    Came for the Pho and really enjoyed it!  We go...
8    Absolutely the most Unique experience in a nai...
9    Wow. I walked in and sat at the bar for 10 min...
Name: text, dtype: object

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [5]:
import spacy 
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore
from spacy.tokenizer import Tokenizer

In [6]:
yelp.shape

(10000, 9)

In [7]:
yelp['text'].head()


0    BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...
1    Came here for lunch Togo. Service was quick. S...
2    I've been to Vegas dozens of times and had nev...
3    We went here on a night where they closed off ...
4    3.5 to 4 stars\n\nNot bad for the price, $12.9...
Name: text, dtype: object

In [8]:
def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True, min_len=4, max_len=20) if token not in STOPWORDS]

In [9]:
yelp['tokens'] = yelp['text'].apply(tokenize)

In [10]:
yelp['tokens'].head()

0    [beware, fake, fake, fake, small, business, al...
1    [came, lunch, togo, service, quick, staff, fri...
2    [vegas, dozens, times, stepped, foot, circus, ...
3    [went, night, closed, street, party, best, act...
4    [stars, price, lunch, seniors, eating, food, s...
Name: tokens, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [11]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object -- also setting max and min on df etc...
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

# Learn vocab and transform data into form we want
sparse = tfidf.fit_transform(yelp['text'])

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

dtm.head()

,able,absolutely,actually,amazing,area,arrived,asked,atmosphere,attentive,away,...,wonderful,work,working,worst,worth,wouldn,wrong,year,years,yelp
0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.130355,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.154235,0.0,0.0
3,0.0,0.0,0.302932,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [12]:
# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [13]:
fake_review = ["""
After finshing our dinner, we heard a loud crash come from the back of the place. The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation, given that I was completley drunk and could not stop laughing. The beer was a little flat, the 'chicken' was a little gamey. 
Service was great though. 4 Stars.
"""]

In [14]:
# Query for simular reveiews
new = tfidf.transform(fake_review)

nn.kneighbors(new.todense())

(array([[1., 1., 1., 1., 1.]]), array([[3543, 6311,  897,  469, 6204]]))

In [15]:
yelp['text'][6304]

'The area within the establishment is split up into two parts: On one half, a bar area which you walk into when entering and on the other half, a sizeable dining area. It is very open concept. The ceilings are very high and it does look more upscale. \n\nI visited on a Friday night, hoping to try a few excellent beers. By the way, the only reason I came was because this place is called "Beerbistro" and I love beer!! Anyways, I had decided on a flight of beers because there were simply too many to choose from. My idea was that of the 3-6 that I try, I will purchase a pint of the one I like to most. \n\nWell...none of them really tasted any good to me. Now, I am not bad at picking out flavours. so it really was unfortunate that all I could taste was either dirt, sand or metal. \n\nIf you are going to go here for beer, I suggest you do the same before purchasing a pint of anything new on tap. You may be lucky enough to find something you love but for me, I just didn\'t have that experienc

In [16]:
yelp['text'][3543]

'$25 for 2 weeks'

In [17]:
yelp['text'][6204]

#Translation: 
"""I came to Las Vegas on a trip, but I was worried that the nail peeled off.
They answered the questions carefully, and there were Japanese people and people who could speak Japanese, so it was a very beautiful and cozy shop. 
I like the nail hachi-san, who was beautiful and polite. 
I asked for an eyelash extension that wasn't planned, and I had a brown color extension that wasn't so much in Japan. 
I would like to ask you for a massage again."""

"I came to Las Vegas on a trip, but I was worried that the nail peeled off.\nThey answered the questions carefully, and there were Japanese people and people who could speak Japanese, so it was a very beautiful and cozy shop. \nI like the nail hachi-san, who was beautiful and polite. \nI asked for an eyelash extension that wasn't planned, and I had a brown color extension that wasn't so much in Japan. \nI would like to ask you for a massage again."

In [18]:
yelp['text'][897]

'The guy asks teenage girls to take scandalous pictures for his Snapchat. Always an uncomfortable environment.'

In [19]:
yelp['text'][6311]

#Translation: 
"""The weather is very hot and I can't eat anything.
Today I ordered a Korean cold noodle soup, and after the meal, I ordered the dessert. 
The smoothie series will not be too sweet. 
I think the store is very careful, including the delicate dish, the condiment hygiene, and the summer. 
Appetizing, this is a good choice, the service staff is also very dedicated and will come often in the future.
"""

"The weather is very hot and I can't eat anything.\nToday I ordered a Korean cold noodle soup, and after the meal, I ordered the dessert. \nThe smoothie series will not be too sweet. \nI think the store is very careful, including the delicate dish, the condiment hygiene, and the summer. \nAppetizing, this is a good choice, the service staff is also very dedicated and will come often in the future.\n"

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()


# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

#Tuning
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  2.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [22]:
grid_search.best_score_

0.5357

In [23]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 10,
 'vect__max_df': 1.0,
 'vect__max_features': 500,
 'vect__min_df': 0.02}

In [24]:
# I can't stop laughing lol
star_pred = grid_search.predict(fake_review)
print(star_pred)

[5]


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [25]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [26]:
id2word = corpora.Dictionary(yelp['tokens'])
id2word.token2id['chef']

696

Create a bag of words representation of the entire corpus

In [27]:
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]

Your LDA model should be ready for estimation: 

In [28]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 3 # You can change this parameter
                  )

In [29]:
lda.print_topics()

[(0,
  '0.012*"food" + 0.010*"place" + 0.010*"good" + 0.009*"great" + 0.008*"service" + 0.008*"time" + 0.007*"like" + 0.004*"love" + 0.004*"nice" + 0.004*"best"'),
 (1,
  '0.012*"good" + 0.012*"great" + 0.010*"place" + 0.009*"food" + 0.008*"like" + 0.007*"time" + 0.007*"service" + 0.004*"best" + 0.004*"order" + 0.004*"nice"'),
 (2,
  '0.012*"place" + 0.012*"food" + 0.009*"good" + 0.009*"service" + 0.008*"time" + 0.007*"great" + 0.007*"like" + 0.004*"love" + 0.004*"best" + 0.004*"went"')]

In [32]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

topics = [' '.join(t[0:5]) for t in words]

In [33]:
for t in topics: 
    print(t)
    print("\n")

food place good great service


good great place food like


place food good service time




Create 1-2 visualizations of the results

In [34]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [37]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

KeyboardInterrupt: 

In [36]:
from gensim.models.coherencemodel import CoherenceModel
stream = yelp['tokens']

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)